# General Transit Feed Specification Realtime (GTFS Realtime)

## Programos naudojančios GTFS realtime duomenis
- Google Maps
- OpenTripPlanner
- OneBusAway
- Bing Maps
- Transit App
- Traze.app
- CityMapper

Sudaryta pagal {cite}`transitwiki_gtfs_realtime_apps`